In [30]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
# Importacion de librerías
import pandas as pd
import numpy as np
# Gestionar las rutas
import utils.paths as path

In [32]:
raw_data_dir = path.make_dir_function("data")
data_proc_dir = raw_data_dir("interim")
parquet_files = data_proc_dir.glob("*.parquet")

In [33]:
# Directorios de los datasets
parquet_df = {}
for parquet_file in parquet_files:
    parquet_df[f"{parquet_file.name.split('.')[0]}"] = pd.read_parquet(parquet_file)

In [34]:
parquet_df["ratings_v1"].columns , parquet_df["ratings_v1"].shape

(Index(['userId', 'score', 'timestamp', 'movieId'], dtype='object'),
 (11013823, 4))

In [35]:
parquet_df["movies_v1"].columns, parquet_df["movies_v1"].shape

(Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
        'release_year', 'rating', 'duration', 'duration_int', 'duration_type',
        'listed_in', 'description'],
       dtype='object'),
 (22998, 14))

In [45]:
# Eliminaremos la columna duration del dataset de ratings
parquet_df["movies_v1"] = parquet_df["movies_v1"].drop("duration", axis=1)

### Dataset para el desarrllo de la API

In [59]:
def merge_movies_ratings_api(df_list):
    # Union del tipo left de movies_v1 con ratings_v1
    df_merged = pd.merge(df_list["movies_v1"], df_list["ratings_v1"][['score', 'movieId']].groupby("movieId").mean().round(1), left_on="show_id", right_on="movieId", how="left")
    # Resetear la numeracion de los índices
    df_merged = df_merged.reset_index(drop=True)
    # Eliminar el movieId
    # df_merged = df_merged.drop(columns="movieId", axis=1)
    return df_merged

In [60]:
api_df = merge_movies_ratings_api(parquet_df)

In [61]:
# Exportación de la data
def save_as_parquet_api_data(df,file_name):
    data_proc_dir = path.make_dir_function("dataset")
    file_path = data_proc_dir(file_name)
    df.to_parquet(f"{file_path}.parquet")

In [62]:
# Exportando la data que será consumido por la API
save_as_parquet_api_data(api_df,"api_data")

### Dataset para el Sistema de recomendación basado el filtro colaborativo

Usaremos la tripla de columnas usario (``userId``), item (``show_id``,``title``) y voto (``score``).

In [78]:
def merge_movies_and_ratings_fc_model(df_list):
    # Union del tipo left de movies_v1 con ratings_v1
    df_merged = pd.merge(df_list['movies_v1'][['show_id','title']], df_list['ratings_v1'][['movieId','score','userId']], left_on="show_id", right_on="movieId", how="inner")
    # Resetear la numeracion de los índices
    df_merged = df_merged.reset_index(drop=True)
    return df_merged

In [79]:
fc_model_df = merge_movies_and_ratings_fc_model(parquet_df)

In [80]:
# Verifiquemos los registros duplicados
duplicated_records_fc_model = fc_model_df.duplicated().sum()
print(f"Hay {duplicated_records_fc_model} registros nulos donde el mismo usuario ha calificado con una puntuación a la misma película o serie")

Hay 18189 registros nulos donde el mismo usuario ha calificado con una puntuación a la misma película o serie


In [81]:
# Remover los registros duplicados
fc_model_df = fc_model_df.drop_duplicates()

In [85]:
# Exportación de la data
def save_as_parquet_fc_model(df: pd.DataFrame, file_name: str):
    data_proc_dir = path.make_dir_function("dataset")
    file_path = data_proc_dir(file_name)
    df.to_parquet(f"{file_path}.parquet")

In [87]:
# Exportando la data que será consumido por el modelo de Sistema de Recomendación Basado en el Filtro Colaborativo
save_as_parquet_fc_model(fc_model_df,"fc_model")